# Fully-Connected Neural Nets

U pokaznom primjeru implementirali smo dvoslojnu neuralnu mrežu i koristili smo CIFAR-10 korpus podataka. Implementacija je bila jednostavna, ali isto tako i nepraktična jer su loss i gradijent računani u jednoj monolitnoj funkciji. Ovaj pristup je prihvatljiv za jednostavnu dvoslojnu mrežu, ali se ne može održati za veće modele. Želimo da napravimo mreže čiji su djelovi nezavisni tako da ih po potrebi možemo poređati u arhitekturu po želji.

U ovoj vježbi napravićemo jednu takvu mrežu. Za svaki sloj imaćemo `forward` i `backward` funkciju. `forward` funkcija će imati za cilj da za primljene ulazne podatke, težine i ostale parametre izračuna kako izlaz tako i `cache` koji će čuvati vrijednosti za prolaz unazad kao što se može vidjeti u kodu:

```python
def layer_forward(x, w):
    """ Kao ulaz prima podatke x i parametre w """
    # radimo neke proračune ...
    z = # ... dobijamo neku vrijednost
    # ponovo radimo proračune ...
    out = # dobijamo konačnu vrijednost

    cache = (x, w, z, out) # Vrijednosti koje su nam potrebne da izračunamo gradijente

    return out, cache
```

Prolaz unazad će uzeti gradijente "odozgo" i `cache` i izračunaće gradijente u odnosu na ulaz i težine:

```python
def layer_backward(dout, cache):
    """
    Kao ulaz prima dout (izvod loss-a u odnosu na izlaz) i cache, i
    računa izvod u odnosu na ulaz.
    """
    # cache smo gore "spakovali" kao tuple pa ga sada otvaramo
    x, w, z, out = cache

    # Koristimo vrijednosti iz cache-a da dobijemo izvode
    dx = # Izvod loss-a po x
    dw = # Izvod loss-a po w

    return dx, dw
```

In [ ]:
#na početku svakog zadatka moramo izvršiti import paketa koji su nam potrebni
import time
import numpy as np
import matplotlib.pyplot as plt
from funkcije.data_utils import get_CIFAR10_data
from funkcije.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from funkcije.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # postavljamo podrazumijevanu veličinu figure
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# pogledati http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ vraća relativnu grešku """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [ ]:
# Učitavanje obrađenih CIFAR10 podataka.

data = get_CIFAR10_data()
for k, v in list(data.items()):
    print(('%s: ' % k, v.shape))

# Afina sloj: foward
Otvorite file `funkcije/layers.py` i implementirajte `affine_forward` funkciju.

Kada završite testirajte se:

In [ ]:
# Ovaj dio koda služi za testiranje funkcije affine_forward
from funkcije.layers import *

num_inputs = 2
input_shape = (4, 5, 6)
output_dim = 3

input_size = num_inputs * np.prod(input_shape)
weight_size = output_dim * np.prod(input_shape)

x = np.linspace(-0.1, 0.5, num = input_size).reshape(num_inputs, *input_shape)
w = np.linspace(-0.2, 0.3, num = weight_size).reshape(np.prod(input_shape), output_dim)
b = np.linspace(-0.3, 0.1, num = output_dim)

out, _ = affine_forward(x, w, b)
correct_out = np.array([[ 1.49834967,  1.70660132,  1.91485297],
                        [ 3.25553199,  3.5141327,   3.77273342]])

# Uporedite vaše rezultate sa našim. Greška bi trebalo da je oko e-9 ili čak manja.
print('Testiranje affine_forward funkcije:')
print('razlika: ', rel_error(out, correct_out))

# Afina sloj: backward
Sada implementirajte `affine_backward` funkciju i uporedite dobijene rezultate sa numeričkim izvodom.

In [ ]:
# Ovaj dio koda služi za testiranje funkcije affine_backward
np.random.seed(231)
x = np.random.randn(10, 2, 3)
w = np.random.randn(6, 5)
b = np.random.randn(5)
dout = np.random.randn(10, 5)

dx_num = eval_numerical_gradient_array(lambda x: affine_forward(x, w, b)[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: affine_forward(x, w, b)[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: affine_forward(x, w, b)[0], b, dout)

_, cache = affine_forward(x, w, b)
dx, dw, db = affine_backward(dout, cache)

# Greška bi trebalo da je e-10 ili manja
print('Testiranje affine_backward funkcije:')
print('dx greška: ', rel_error(dx_num, dx))
print('dw greška: ', rel_error(dw_num, dw))
print('db greška: ', rel_error(db_num, db))

# ReLU aktivacija: forward

Implementirajte prolaz unaprijed za ReLU aktivacionu funkciju u `relu_forward` i testirajte svoju implementaciju koristeći sledeći kod:

In [ ]:
# Ovaj dio koda služi za testiranje funkcije relu_forward

x = np.linspace(-0.5, 0.5, num = 12).reshape(3, 4)

out, _ = relu_forward(x)
correct_out = np.array([[ 0.,          0.,          0.,          0.,        ],
                        [ 0.,          0.,          0.04545455,  0.13636364,],
                        [ 0.22727273,  0.31818182,  0.40909091,  0.5,       ]])

# Uporedite rezultate sa našim. Greška bi trebalo da je reda e-8
print('Testiranje relu_forward funkcije:')
print('razlika: ', rel_error(out, correct_out))

# ReLU aktivacija: backward
Sada implementirajte prolazak unazad za ReLU aktivacionu funkciju u `relu_backward` i testirajte svoju implementaciju koristeći sledeći kod:

In [ ]:
np.random.seed(231)
x = np.random.randn(10, 10)
dout = np.random.randn(*x.shape)

dx_num = eval_numerical_gradient_array(lambda x: relu_forward(x)[0], x, dout)

_, cache = relu_forward(x)
dx = relu_backward(dout, cache)

# Greška bi trebalo da je reda e-12
print('Testiranje relu_backward funkcije:')
print('dx greška: ', rel_error(dx_num, dx))

# "Sendvič" slojevi
Postoje šabloni slojeva koji se često koriste u neuralnim mrežama. Na primjer, posle afina slojeva često dolazi ReLU nelinearnost. Kako bi ovu učestanost iskoristili, definisaćemo par slojeva u `funkcije/layer_utils.py`.

Za sad bacite pogleda na `affine_relu_forward` i `affine_relu_backward` funkcije, i pokrenite sledeći kod kako biste u prolazu unazad provjerili gradijente:

In [ ]:
from funkcije.layer_utils import affine_relu_forward, affine_relu_backward
np.random.seed(231)
x = np.random.randn(2, 3, 4)
w = np.random.randn(12, 10)
b = np.random.randn(10)
dout = np.random.randn(2, 10)

out, cache = affine_relu_forward(x, w, b)
dx, dw, db = affine_relu_backward(dout, cache)

dx_num = eval_numerical_gradient_array(lambda x: affine_relu_forward(x, w, b)[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: affine_relu_forward(x, w, b)[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: affine_relu_forward(x, w, b)[0], b, dout)

# Greška bi trebalo da je oko e-10 ili manja
print('Testiranje affine_relu_forward i affine_relu_backward:')
print('dx greška: ', rel_error(dx_num, dx))
print('dw greška: ', rel_error(dw_num, dw))
print('db greška: ', rel_error(db_num, db))

# Loss slojevi: Softmax i SVM

Ove funkcije smo za vas implementirali u pokaznom primjeru pa vam ih dajemo i ovdje. Pogledajte kako funkcionišu (ukoliko niste sigurni) gledanjem `funkcije/layers.py`.

Pokrenite kod kako biste se uvjerili da sve radi kako treba:

In [ ]:
np.random.seed(231)
num_classes, num_inputs = 10, 50
x = 0.001 * np.random.randn(num_inputs, num_classes)
y = np.random.randint(num_classes, size=num_inputs)

dx_num = eval_numerical_gradient(lambda x: svm_loss(x, y)[0], x, verbose=False)
loss, dx = svm_loss(x, y)

# Testirajte svm_loss funkciju. Loss bi trebalo da je oko 9, a dx greška reda e-9
print('Testing svm_loss:')
print('loss: ', loss)
print('dx error: ', rel_error(dx_num, dx))

dx_num = eval_numerical_gradient(lambda x: softmax_loss(x, y)[0], x, verbose=False)
loss, dx = softmax_loss(x, y)

# Testirajte softmax_loss funkciju. Loss bi trebalo da je oko 2.3, a dx greška reda e-8
print('\Testiranje softmax_loss:')
print('loss: ', loss)
print('dx greška: ', rel_error(dx_num, dx))

# Dvoslojna mreža

U pokaznom primjeru implementirali smo dvoslojnu mrežu u jednoj monolitnoj klasi. Sada vi za zadatak imate to da učinite korištenjem funkcija koje ste napravili u ovom zadatku.

Otvorite `funkcije/fc_net.py` i završite implementaciju `TwoLayerNet` klase. Ova klasa će služiti kao model za ostale mreže koje ćete implementirati u ovom zadatku. Kod koji slijedi omogućava vam testiranje klase koju ste implementirali:

In [ ]:
from funkcije.fc_net import *
np.random.seed(231)
N, D, H, C = 3, 5, 50, 7
X = np.random.randn(N, D)
y = np.random.randint(C, size=N)

std = 1e-3
model = TwoLayerNet(input_dim = D, hidden_dim = H, num_classes = C, weight_scale = std)

print('Testiranje inicijalizacije ... ')
W1_std = abs(model.params['W1'].std() - std)
b1 = model.params['b1']
W2_std = abs(model.params['W2'].std() - std)
b2 = model.params['b2']
assert W1_std < std / 10, 'Težine prvog sloja ne izgledaju dobro'
assert np.all(b1 == 0), 'Biasi prvog sloja ne izgledaju dobro'
assert W2_std < std / 10, 'Težine drugog sloja ne izgledaju dobro'
assert np.all(b2 == 0), 'Biasi drugog sloja ne izgledaju dobro'

print('Testiranje test-time prolaza unaprijed ... ')
model.params['W1'] = np.linspace(-0.7, 0.3, num = D*H).reshape(D, H)
model.params['b1'] = np.linspace(-0.1, 0.9, num = H)
model.params['W2'] = np.linspace(-0.3, 0.4, num = H*C).reshape(H, C)
model.params['b2'] = np.linspace(-0.9, 0.1, num = C)
X = np.linspace(-5.5, 4.5, num = N*D).reshape(D, N).T
scores = model.loss(X)
correct_scores = np.asarray(
  [[11.53165108,  12.2917344,   13.05181771,  13.81190102,  14.57198434, 15.33206765,  16.09215096],
   [12.05769098,  12.74614105,  13.43459113,  14.1230412,   14.81149128, 15.49994135,  16.18839143],
   [12.58373087,  13.20054771,  13.81736455,  14.43418138,  15.05099822, 15.66781506,  16.2846319 ]])
scores_diff = np.abs(scores - correct_scores).sum()
assert scores_diff < 1e-6, 'Problemi sa test-time prolazom unaprijed'

print('Testiranje trening loss-a (bez regularizacije)')
y = np.asarray([0, 5, 1])
loss, grads = model.loss(X, y)
correct_loss = 3.4702243556
assert abs(loss - correct_loss) < 1e-10, 'Problemi sa training-time loss-om'

model.reg = 1.0
loss, grads = model.loss(X, y)
correct_loss = 26.5948426952
assert abs(loss - correct_loss) < 1e-10, 'Problemi sa regularizacijom'

# Greške bi trebalo da su oko e-7 ili manje
for reg in [0.0, 0.7]:
    print('Provjeravanje izvoda sa regularizacijom = ', reg)
    model.reg = reg
    loss, grads = model.loss(X, y)

    for name in sorted(grads):
        f = lambda _: model.loss(X, y)[0]
        grad_num = eval_numerical_gradient(f, model.params[name], verbose=False)
        print('%s relativna greška: %.2e' % (name, rel_error(grad_num, grads[name])))

# Solver

U pokaznom primjeru je logika treniranja implementirana unutar modela. Praktičnije rješenje iziskuje da se ove dvije stvari razdvoje u različite klase.

Otvorite `funkcije/solver.py`. Iskoristite `Solver` instancu da istrenirate `TwoLayerNet` kako biste dobili model koji ima najmanje `50%` tačnost na validacionom setu.

In [ ]:
model = TwoLayerNet()
solver = None

##############################################################################
# URADITI: Iskoristite Solver instancu da istrenirate TwoLayerNet kako biste    #
# dobili model koji ima najmanje 50% tačnost na validacionom setu.           #
##############################################################################
pass
##############################################################################
#                             KRAJ VAŠEG KODA                                #
##############################################################################

In [ ]:
# Pokrenite ovaj kod kako biste vizuelizovali trening loss i trening / validacija preciznost

plt.subplot(2, 1, 1)
plt.title('Trening loss')
plt.plot(solver.loss_history, 'o')
plt.xlabel('Iteracija')

plt.subplot(2, 1, 2)
plt.title('Preciznost')
plt.plot(solver.train_acc_history, '-o', label = 'train')
plt.plot(solver.val_acc_history, '-o', label = 'val')
plt.plot([0.5] * len(solver.val_acc_history), 'k--')
plt.xlabel('Epoha')
plt.legend(loc = 'lower right')
plt.gcf().set_size_inches(15, 12)
plt.show()

# Višeslojna mreža

Sledeći zadatak jeste da napravimo potpuno povezanu mrežu sa proizvoljnim brojem skrivenih slojeva.

Pročitajte sve iz `FullyConnectedNet` klase u `funkcije/fc_net.py`.

Implementirajte inicijalizaciju, prolaz unaprijed i prolaz unazad.

## Inicijalni loss i provjera gradijenta

Provjerite inicijalne vrijednosti mreže sa i bez regularizacije. Za provjeru gradijenata treba očekivati greške reda 1e-7 ili manje.

In [ ]:
np.random.seed(231)
N, D, H1, H2, C = 2, 15, 20, 30, 10
X = np.random.randn(N, D)
y = np.random.randint(C, size=(N,))

for reg in [0, 3.14]:
    print('Provjera sa regularizacijom = ', reg)
    model = FullyConnectedNet([H1, H2], input_dim = D, num_classes = C,
                            reg = reg, weight_scale = 5e-2, dtype = np.float64)

    loss, grads = model.loss(X, y)
    print('Inicijalni loss: ', loss)
  
  # Većina grešaka bi trebalo da je reda e-7 ili manje.   
  # NAPOMENA: Za W2 greška može biti reda e-5
  # za provjeru kada je reg = 0.0
    for name in sorted(grads):
        f = lambda _: model.loss(X, y)[0]
        grad_num = eval_numerical_gradient(f, model.params[name], verbose = False, h = 1e-5)
        print('%s relativna greška: %.2e' % (name, rel_error(grad_num, grads[name])))

Za provjeru pokušajte da overfitujete mali korpus podataka od 50 slika. Prvo pokušavamo sa troslojnom mrežom u kojoj skriveni slojevi imaju po 100 neurona. Pokušajte da mijenjate learning_rate i weight_scale kako biste dobili 100% preciznost na trening setu nakon najviše 20 epoha.

In [ ]:
num_train = 50
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

weight_scale = 1e-2
learning_rate = 1e-4
model = FullyConnectedNet([100, 100],
              weight_scale = weight_scale, dtype = np.float64)
solver = Solver(model, small_data,
                print_every = 10, num_epochs = 20, batch_size = 25,
                update_rule = 'sgd',
                optim_config = {
                  'learning_rate': learning_rate,
                }
         )
solver.train()

plt.plot(solver.loss_history, 'o')
plt.title('Vrijednosti trening loss-a')
plt.xlabel('Iteracija')
plt.ylabel('Trening loss')
plt.show()

Sada pokušajte isto sa petoslojnom mrežom u kojoj svaki sloj ima 100 neurona. 

In [ ]:
num_train = 50
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

learning_rate = 2e-3
weight_scale = 1e-5
model = FullyConnectedNet([100, 100, 100, 100],
                weight_scale = weight_scale, dtype = np.float64)
solver = Solver(model, small_data,
                print_every = 10, num_epochs = 20, batch_size = 25,
                update_rule = 'sgd',
                optim_config = {
                  'learning_rate': learning_rate,
                }
         )
solver.train()

plt.plot(solver.loss_history, 'o')
plt.title('Vrijednosti trening loss-a')
plt.xlabel('Iteracija')
plt.ylabel('Trening loss')
plt.show()

# Pravila za ažuriranje parametara
Za sada smo koristili samo `vanila` SGD. Kao što je i na predavanju objašnjeno, postoje mnogo bolji načini optimizacije i neke ćemo implementirati. 

# SGD+Momentum
Otvorite `funkcije/optim.py` i implementirajte the SGD+momentum pravilo ažuriranja u funkciji `sgd_momentum` i pokrenite kod koji slijedi kako biste provjerili rezultate. Trebalo bi da je greška manja od e-8.

In [ ]:
from cs231n.optim import sgd_momentum

N, D = 4, 5
w = np.linspace(-0.4, 0.6, num=N*D).reshape(N, D)
dw = np.linspace(-0.6, 0.4, num=N*D).reshape(N, D)
v = np.linspace(0.6, 0.9, num=N*D).reshape(N, D)

config = {'learning_rate': 1e-3, 'velocity': v}
next_w, _ = sgd_momentum(w, dw, config=config)

expected_next_w = np.asarray([
  [ 0.1406,      0.20738947,  0.27417895,  0.34096842,  0.40775789],
  [ 0.47454737,  0.54133684,  0.60812632,  0.67491579,  0.74170526],
  [ 0.80849474,  0.87528421,  0.94207368,  1.00886316,  1.07565263],
  [ 1.14244211,  1.20923158,  1.27602105,  1.34281053,  1.4096    ]])
expected_velocity = np.asarray([
  [ 0.5406,      0.55475789,  0.56891579, 0.58307368,  0.59723158],
  [ 0.61138947,  0.62554737,  0.63970526,  0.65386316,  0.66802105],
  [ 0.68217895,  0.69633684,  0.71049474,  0.72465263,  0.73881053],
  [ 0.75296842,  0.76712632,  0.78128421,  0.79544211,  0.8096    ]])

print('next_w greška: ', rel_error(next_w, expected_next_w))
print('velocity greška: ', rel_error(expected_velocity, config['velocity']))

Kada ste završili sa prethodnim pokrenite sledeći kod kako biste istrenirali šestoslojnu mrežu sa SGD i sa SGD+momentum. Očekujemo da SGD+momentum brže iskonvergira.

In [ ]:
num_train = 4000
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

solvers = {}

for update_rule in ['sgd', 'sgd_momentum']:
    print('radimo sa ', update_rule)
    model = FullyConnectedNet([100, 100, 100, 100, 100], weight_scale = 5e-2)

    solver = Solver(model, small_data,
                  num_epochs = 5, batch_size = 100,
                  update_rule = update_rule,
                  optim_config = {
                    'learning_rate': 1e-2,
                  },
                  verbose = True)
    solvers[update_rule] = solver
    solver.train()
    print()

plt.subplot(3, 1, 1)
plt.title('Trening loss')
plt.xlabel('Iteracija')

plt.subplot(3, 1, 2)
plt.title('Trening preciznost')
plt.xlabel('Epoha')

plt.subplot(3, 1, 3)
plt.title('Validacija preciznost')
plt.xlabel('Epoha')

for update_rule, solver in list(solvers.items()):
    plt.subplot(3, 1, 1)
    plt.plot(solver.loss_history, 'o', label = update_rule)

    plt.subplot(3, 1, 2)
    plt.plot(solver.train_acc_history, '-o', label = update_rule)

    plt.subplot(3, 1, 3)
    plt.plot(solver.val_acc_history, '-o', label = update_rule)

for i in [1, 2, 3]:
    plt.subplot(3, 1, i)
    plt.legend(loc = 'upper center', ncol = 4)
plt.gcf().set_size_inches(15, 15)
plt.show()

# RMSProp i Adam
RMSProp [1] i Adam [2] su pravila ažuriranja parametara koje smo pomenuli na časovima, a koja su detaljnije objašnjena u priloženim radovima.

U fajlu `funkcije/optim.py`, implementirajte RMSProp u `rmsprop` funkciji i implementirajte Adam u `adam` funkciji, te provjerite vašu implementaciju u testu koji slijedi. 

**NAPOMENA:** Za zadatak treba implementirati kompletno Adamovo pravilo (sa mehanizmom za korekciju biasa), a ne onu verziju koja je prikazana na času. Detaljnije u radu.

[1] Tijmen Tieleman and Geoffrey Hinton. "Lecture 6.5-rmsprop: Divide the gradient by a running average of its recent magnitude." COURSERA: Neural Networks for Machine Learning 4 (2012).

[2] Diederik Kingma and Jimmy Ba, "Adam: A Method for Stochastic Optimization", ICLR 2015.

In [ ]:
from funkcije.optim import rmsprop

N, D = 4, 5
w = np.linspace(-0.4, 0.6, num=N*D).reshape(N, D)
dw = np.linspace(-0.6, 0.4, num=N*D).reshape(N, D)
cache = np.linspace(0.6, 0.9, num=N*D).reshape(N, D)

config = {'learning_rate': 1e-2, 'cache': cache}
next_w, _ = rmsprop(w, dw, config=config)

expected_next_w = np.asarray([
  [-0.39223849, -0.34037513, -0.28849239, -0.23659121, -0.18467247],
  [-0.132737,   -0.08078555, -0.02881884,  0.02316247,  0.07515774],
  [ 0.12716641,  0.17918792,  0.23122175,  0.28326742,  0.33532447],
  [ 0.38739248,  0.43947102,  0.49155973,  0.54365823,  0.59576619]])
expected_cache = np.asarray([
  [ 0.5976,      0.6126277,   0.6277108,   0.64284931,  0.65804321],
  [ 0.67329252,  0.68859723,  0.70395734,  0.71937285,  0.73484377],
  [ 0.75037008,  0.7659518,   0.78158892,  0.79728144,  0.81302936],
  [ 0.82883269,  0.84469141,  0.86060554,  0.87657507,  0.8926    ]])

# Očekujemo grešku manju od e-7
print('next_w greška: ', rel_error(expected_next_w, next_w))
print('cache greška: ', rel_error(expected_cache, config['cache']))

In [ ]:
from funkcije.optim import adam

N, D = 4, 5
w = np.linspace(-0.4, 0.6, num=N*D).reshape(N, D)
dw = np.linspace(-0.6, 0.4, num=N*D).reshape(N, D)
m = np.linspace(0.6, 0.9, num=N*D).reshape(N, D)
v = np.linspace(0.7, 0.5, num=N*D).reshape(N, D)

config = {'learning_rate': 1e-2, 'm': m, 'v': v, 't': 5}
next_w, _ = adam(w, dw, config=config)

expected_next_w = np.asarray([
  [-0.40094747, -0.34836187, -0.29577703, -0.24319299, -0.19060977],
  [-0.1380274,  -0.08544591, -0.03286534,  0.01971428,  0.0722929],
  [ 0.1248705,   0.17744702,  0.23002243,  0.28259667,  0.33516969],
  [ 0.38774145,  0.44031188,  0.49288093,  0.54544852,  0.59801459]])
expected_v = np.asarray([
  [ 0.69966,     0.68908382,  0.67851319,  0.66794809,  0.65738853,],
  [ 0.64683452,  0.63628604,  0.6257431,   0.61520571,  0.60467385,],
  [ 0.59414753,  0.58362676,  0.57311152,  0.56260183,  0.55209767,],
  [ 0.54159906,  0.53110598,  0.52061845,  0.51013645,  0.49966,   ]])
expected_m = np.asarray([
  [ 0.48,        0.49947368,  0.51894737,  0.53842105,  0.55789474],
  [ 0.57736842,  0.59684211,  0.61631579,  0.63578947,  0.65526316],
  [ 0.67473684,  0.69421053,  0.71368421,  0.73315789,  0.75263158],
  [ 0.77210526,  0.79157895,  0.81105263,  0.83052632,  0.85      ]])

# Očekujemo grešku manju od e-7
print('next_w greška: ', rel_error(expected_next_w, next_w))
print('v greška: ', rel_error(expected_v, config['v']))
print('m greška: ', rel_error(expected_m, config['m']))

Nakon uspješne implementacije ova dva pravila, istrenirajte sledeću mrežu kako biste uočili razlike:

In [ ]:
learning_rates = {'rmsprop': 1e-4, 'adam': 1e-3}
for update_rule in ['adam', 'rmsprop']:
    print('running with ', update_rule)
    model = FullyConnectedNet([100, 100, 100, 100, 100], weight_scale = 5e-2)

    solver = Solver(model, small_data,
                  num_epochs = 5, batch_size = 100,
                  update_rule = update_rule,
                  optim_config = {
                    'learning_rate': learning_rates[update_rule]
                  },
                  verbose = True)
    solvers[update_rule] = solver
    solver.train()
    print()

plt.subplot(3, 1, 1)
plt.title('Trening loss')
plt.xlabel('Iteracija')

plt.subplot(3, 1, 2)
plt.title('Trening preciznost')
plt.xlabel('Epoha')

plt.subplot(3, 1, 3)
plt.title('Validacija preciznost')
plt.xlabel('Epoha')

for update_rule, solver in list(solvers.items()):
    plt.subplot(3, 1, 1)
    plt.plot(solver.loss_history, 'o', label = update_rule)

    plt.subplot(3, 1, 2)
    plt.plot(solver.train_acc_history, '-o', label = update_rule)

    plt.subplot(3, 1, 3)
    plt.plot(solver.val_acc_history, '-o', label = update_rule)

for i in [1, 2, 3]:
    plt.subplot(3, 1, i)
    plt.legend(loc = 'upper center', ncol = 4)
plt.gcf().set_size_inches(15, 15)
plt.show()

# Oprobajte se!

Pokušajte da napravite najbolji mogući model na korpusu podataka CIFAR-10 i sačuvajte ga u `best_model`.

Takođe, prije pokretanja ovog koda možete sačekati da riješite `BatchNormalization.ipynb` i `Dropout.ipynb` jer značajno pomažu mreži da ostvari bolje rezultate.


In [ ]:
best_model = None
################################################################################
# URADITI: Napravite najbolji mogući model i sačuvajte ga u best_model         #
################################################################################
pass
################################################################################
#                              KRAJ VAŠEG KODA                                 #
################################################################################

# Testirajte model!
Pokrenite vaš model na validacionom i test setu. Očekujemo da ostvarite do 55% preciznosti.

In [ ]:
y_test_pred = np.argmax(best_model.loss(data['X_test']), axis = 1)
y_val_pred = np.argmax(best_model.loss(data['X_val']), axis = 1)
print('Preciznost na validacionom setu: ', (y_val_pred == data['y_val']).mean())
print('Preciznost na test setu: ', (y_test_pred == data['y_test']).mean())